In [12]:
import types
import functools
import itertools
import logging

def vector_map_wrapper(func, var):
    @functools.wraps(func)
    def _vec_map_wrapper(*args):
        # applies the given function to all self.var
        # todo: apply arguments to other
        wrapped_objs = getattr(args[0], var)
        logging.warning("Wrapping {} with value {}".format(func, wrapped_objs))
        return list(map(func, getattr(args[0], var)))

    return _vec_map_wrapper


class MetaChannelGrouper(type):

    def __new__(meta, classname, bases, namespace, **kwargs):
        new_namespace = {}
        for b in bases:
            print("Found base {}".format(b))
            print(b.__dict__)
        vector_variable = kwargs.get("vector_variable", "channels")
        for attribute_name, attribute in namespace.items():
            logging.warning("Found %s attribute", attribute_name)
            if isinstance(attribute, types.FunctionType) and attribute_name != "__init__":
                # replace it with a wrapped version
                logging.warning("Wrapping {}".format(attribute_name))
                
                attribute = vector_map_wrapper(attribute, vector_variable)
            new_namespace[attribute_name] = attribute
        new_class = type.__new__(meta, classname, bases, new_namespace)
        print(new_class.sqr)
        print(super(new_class.__class__, new_class))
        
        return new_class

class MetaChannelGrouper2(type):
    def __new__(meta, classname, bases, namespace, **kwargs):
        print("\n\nMetaChannelGrouper2")
        new_namespace= {}
        vector_variable = kwargs.get("vector_variable", "channels")
        for b in bases:
            print(b.__dict__)
        
        items = itertools.chain.from_iterable(b.__dict__.items() for b in bases)
        print(items)
        for attribute_name, attribute in items:
            logging.warning("Found %s attribute %s", attribute_name, attribute)
            if isinstance(attribute, types.FunctionType) and attribute_name != "__init__":
                # replace function with vectorized version
                logging.warning("Wrapping %s", attribute_name)
                
                attribute = vector_map_wrapper(attribute, vector_variable)
            new_namespace[attribute_name] = attribute
            
        new_class = type.__new__(meta, classname, bases, new_namespace)
        print("Created class with attrs: {}".format(new_class.__dict__))
        for b in bases:
            print("new_class is instanceof({}): {}".format(b, isinstance(new_class, b)))
        return new_class


logging.basicConfig(level=logging.DEBUG)

In [21]:
import numpy as np
import traceback as tb
class TestSimple:
    def __init__(self, val):
        self.val = val
    def sqr(self):
        return self.val ** 2

class TestVec1(TestSimple, metaclass=MetaChannelGrouper):
    channels = [TestSimple(1), TestSimple(4), TestSimple(9)]
    
    def __init__(self):
        pass

class TestVec2(TestSimple, metaclass=MetaChannelGrouper2):
    channels = [TestSimple(1), TestSimple(4), TestSimple(9)]
    
    def __init__(self):
        pass

Found base <class '__main__.TestSimple'>
{'__dict__': <attribute '__dict__' of 'TestSimple' objects>, '__weakref__': <attribute '__weakref__' of 'TestSimple' objects>, '__init__': <function TestSimple.__init__ at 0x00000146B0BF7730>, 'sqr': <function TestSimple.sqr at 0x00000146B0BF7400>, '__doc__': None, '__module__': '__main__'}
<function TestSimple.sqr at 0x00000146B0BF7400>
<super: <class 'MetaChannelGrouper'>, <MetaChannelGrouper object>>


MetaChannelGrouper2
{'__dict__': <attribute '__dict__' of 'TestSimple' objects>, '__weakref__': <attribute '__weakref__' of 'TestSimple' objects>, '__init__': <function TestSimple.__init__ at 0x00000146B0BF7730>, 'sqr': <function TestSimple.sqr at 0x00000146B0BF7400>, '__doc__': None, '__module__': '__main__'}
Created class with attrs: {'__dict__': <attribute '__dict__' of 'TestSimple' objects>, '__weakref__': <attribute '__weakref__' of 'TestSimple' objects>, '__init__': <function TestSimple.__init__ at 0x00000146B0BF7730>, 'sqr': <function Te

In [22]:
t1 = TestVec1()
print(isinstance(t1, TestSimple))

try:
    t2 = TestVec2()
    print(isinstance(t2, TestSimple))
except TypeError:
    tb.print_exc()

True


Traceback (most recent call last):
  File "<ipython-input-22-28a536886796>", line 5, in <module>
    t2 = TestVec2()
TypeError: __init__() missing 1 required positional argument: 'val'


In [24]:
class VectorWrapper(object):
    # assumes all objects have same methods as first
    def __init__(self, wrapped_objs):
        self.wrapped_objs = wrapped_objs
        
    def __getattr__(self, attr):
        orig_attr = self.wrapped_objs[0].__getattribute__(attr)
        if callable(orig_attr):
            def hooked(*args, **kwargs):
                self.pre()
                
                result = [wo.__getattribute__(attr)(*args, **kwargs) for wo in self.wrapped_objs]
                # prevent wrapped class from becoming unwrapped
#                 if result == self.wrapped_class:
#                     return self
                self.post()
                return result
            return hooked
        else:
            return orig_attr
        
    def pre(self):
        print(">> pre")
    
    def post(self):
        print("<< post")

t3 = VectorWrapper([TestSimple(1), TestSimple(3), TestSimple(5)])
t3.sqr()

>> pre
<< post


[1, 9, 25]

In [3]:
print([c.val for c in TestVec.channels])
c = TestVec1()

c.sqr(c.channels)

NameError: name 'TestVec' is not defined

In [ ]:
print(c.__class__)
g

In [39]:
raise NameError()

NameError: 